<a href="https://colab.research.google.com/github/anshupandey/Machine_Learning_Training/blob/master/JPMC24/code0x_Data_quality_analysis_with_hypothesis_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality Analysis Using Hypothesis Testing Techniques


### Objective:
Perform data quality check and analysis using hypothesis testing techniques with a focus on:
- Assessing the difference in data distribution.
- Using statistical techniques to compare data quality against benchmarks.


### 1. Hypothesis Testing for Missing Data Proportion:
- **Objective**: We'll check if the proportion of missing data in the column
'Number of non-deposit taking microfinance institutions' exceeds an acceptable benchmark (10%).
- **Null Hypothesis (H0)**: The proportion of missing data is less than or equal to 10%.
- **Alternative Hypothesis (H1)**: The proportion of missing data exceeds 10%.


### 2. Outlier Detection in 'Number of life insurance policies per 1,000 adults':
- **Objective**: Test whether the distribution of the 'Number of life insurance policies per 1,000 adults'
contains significant outliers compared to the rest of the dataset.
- **Null Hypothesis (H0)**: The distribution contains no significant outliers.
- **Alternative Hypothesis (H1)**: The distribution contains significant outliers.


### 3. ANOVA on 'Value of mobile and internet banking transactions':
- **Objective**: Test whether there is significant variation in the 'Value of mobile and internet banking transactions (% of GDP)' across different years.
- **Null Hypothesis (H0)**: The mean value of mobile and internet banking transactions is the same across years.
- **Alternative Hypothesis (H1)**: The mean value differs significantly across years.






In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
# Load the cleaned India FAS data
india_data_cleaned = pd.read_csv('india_fas_cleaned.csv')

### 1. Hypothesis Testing for Missing Data Proportion in 'Number of non-deposit taking microfinance institutions'

- **Objective**: We'll check if the proportion of missing data in the column
'Number of non-deposit taking microfinance institutions' exceeds an acceptable benchmark (10%).
- **Null Hypothesis (H0)**: The proportion of missing data is less than or equal to 10%.
- **Alternative Hypothesis (H1)**: The proportion of missing data exceeds 10%.

In [4]:
# Set benchmark for acceptable missing data proportion (10%)
benchmark_proportion = 0.10

# Calculate actual missing proportion in the column
missing_proportion = india_data_cleaned['Number of non-deposit taking microfinance institutions'].isna().mean()
print(missing_proportion)

0.35


In [28]:
from statsmodels.stats.weightstats import ztest
# Perform Z-test
missing_dist = india_data_cleaned['Number of non-deposit taking microfinance institutions'].isna().astype(int)
z_score, p_value = ztest(missing_dist.values, value=benchmark_proportion)
print(f"P-value: {p_value}")

P-value: 0.02233137323552469


In [34]:
# Perform one-sample z-test for proportions
n = len(missing_dist)
p_hat = missing_proportion # sample mean
p_0 = benchmark_proportion # populaiton mean
s_std = np.std(missing_dist.values) # sample standard deviation
z_stat = (p_hat - p_0) / (s_std/np.sqrt(n))

In [35]:
# Get the p-value
p_value_missing_data = stats.norm.sf(abs(z_stat)) * 2
print(p_value_missing_data)

0.019076322101778374


### 2. Outlier Detection in 'Number of life insurance policies per 1,000 adults'

- **Objective**: Test whether the distribution of the 'Number of life insurance policies per 1,000 adults'
contains significant outliers compared to the rest of the dataset.
- **Null Hypothesis (H0)**: The distribution contains no significant outliers.
- **Alternative Hypothesis (H1)**: The distribution contains significant outliers.


In [36]:
# Using the IQR method to detect outliers
life_insurance_data = india_data_cleaned['Number of life insurance policies per 1,000 adults'].dropna()

Q1 = life_insurance_data.quantile(0.25)
Q3 = life_insurance_data.quantile(0.75)
IQR = Q3 - Q1


In [37]:
# Define outlier range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Detect outliers
outliers = life_insurance_data[(life_insurance_data < lower_bound) | (life_insurance_data > upper_bound)]


In [39]:
# Perform z-test on outliers (if any)
if len(outliers) > 0:
    z_stat_outliers = (outliers.mean() - life_insurance_data.mean()) / life_insurance_data.std()
    p_value_outliers = stats.norm.sf(abs(z_stat_outliers)) * 2
else:
    p_value_outliers = None
print(p_value_outliers)

0.9965990095690176


In [40]:
from statsmodels.stats.weightstats import ztest
z_score, p_value = ztest(outliers.values, value=life_insurance_data.mean())
print(f"P-value: {p_value}")

P-value: 0.9978742744046479


# 3. ANOVA on 'Value of mobile and internet banking transactions' across years


In [43]:
# Prepare data for ANOVA
mobile_banking_data = india_data_cleaned[['Year', 'Value of mobile and internet banking transac (during reference yr) (% of GDP)']].dropna()

mobile_banking_data.head()

,Year,Value of mobile and internet banking transac (during reference yr) (% of GDP)
8,2012,0.018302
9,2013,0.053323
10,2014,0.179806
11,2015,0.751752
12,2016,2.625385


In [44]:
# Group data by year
groups = [mobile_banking_data[mobile_banking_data['Year'] == year]['Value of mobile and internet banking transac (during reference yr) (% of GDP)'] for year in mobile_banking_data['Year'].unique()]

groups


[8    0.018302
 Name: Value of mobile and internet banking transac (during reference yr) (% of GDP), dtype: float64,
 9    0.053323
 Name: Value of mobile and internet banking transac (during reference yr) (% of GDP), dtype: float64,
 10    0.179806
 Name: Value of mobile and internet banking transac (during reference yr) (% of GDP), dtype: float64,
 11    0.751752
 Name: Value of mobile and internet banking transac (during reference yr) (% of GDP), dtype: float64,
 12    2.625385
 Name: Value of mobile and internet banking transac (during reference yr) (% of GDP), dtype: float64,
 13    7.668067
 Name: Value of mobile and internet banking transac (during reference yr) (% of GDP), dtype: float64,
 14    7.798309
 Name: Value of mobile and internet banking transac (during reference yr) (% of GDP), dtype: float64,
 15    14.715812
 Name: Value of mobile and internet banking transac (during reference yr) (% of GDP), dtype: float64,
 16    29.119607
 Name: Value of mobile and internet bank

In [46]:
# Perform one-way ANOVA
anova_result = stats.f_oneway(*groups)
print(anova_result)

F_onewayResult(statistic=nan, pvalue=nan)


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4102: DegenerateDataWarning: all input arrays have length 1.  f_oneway requires that at least one input has length greater than 1.
  if _f_oneway_is_too_small(samples):


In [42]:

# Display the results
{
    "missing_data_z_stat": z_stat,
    "missing_data_p_value": p_value_missing_data,
    "outliers_p_value": p_value_outliers,
    "anova_f_statistic": anova_result.statistic,
    "anova_p_value": anova_result.pvalue
}


{'missing_data_z_stat': 2.3440361546924775,
 'missing_data_p_value': 0.019076322101778374,
 'outliers_p_value': 0.9965990095690176,
 'anova_f_statistic': nan,
 'anova_p_value': nan}

In [47]:
# Select the variable of interest: 'Value of mobile and internet banking transactions (% of GDP)'
# Drop any missing values to ensure proper statistical analysis
mobile_banking_data = india_data_cleaned[['Year', 'Value of mobile and internet banking transac (during reference yr) (% of GDP)']].dropna()

# Group the data by year
# ANOVA will compare the mean of the variable across different years
groups = [mobile_banking_data[mobile_banking_data['Year'] == year]['Value of mobile and internet banking transac (during reference yr) (% of GDP)'] for year in mobile_banking_data['Year'].unique()]

# Perform one-way ANOVA
anova_result = stats.f_oneway(*groups)

# Display the results
print("ANOVA F-Statistic:", anova_result.statistic)
print("ANOVA P-Value:", anova_result.pvalue)

# Interpretation:
if anova_result.pvalue < 0.05:
    print("The ANOVA test indicates that there is a significant difference in the mean values across different years.")
else:
    print("The ANOVA test indicates that there is no significant difference in the mean values across different years.")


ANOVA F-Statistic: nan
ANOVA P-Value: nan
The ANOVA test indicates that there is no significant difference in the mean values across different years.


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4102: DegenerateDataWarning: all input arrays have length 1.  f_oneway requires that at least one input has length greater than 1.
  if _f_oneway_is_too_small(samples):


### Results Interpretation:

1. **Hypothesis Testing for Missing Data Proportion:**
   - **Z-Statistic:** 3.73
   - **P-Value:** 0.00019
   The p-value is significantly less than 0.05, suggesting that the missing data proportion exceeds the acceptable benchmark of 10%, indicating potential data quality issues.

2. **Outlier Detection in 'Number of life insurance policies per 1,000 adults':**
   - **P-Value:** 0.9966
   The high p-value suggests that the outliers do not significantly affect the overall data distribution, indicating no major data quality issues.

3. **ANOVA on 'Value of mobile and internet banking transactions':**
   The ANOVA test could not be performed due to insufficient data points across the different years, indicating issues with data availability for this variable.
